<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguaje Natural

## Taller #9: Modelado de temas
`Fecha de entrega: Mayo 6, 2021. (Antes del inicio de la próxima clase).`

`Modo de entrega: Subir link de GitHub al aula virtual.`

In [6]:
import re
import pandas as pd 
from pprint import pprint

from nltk.corpus import stopwords
stopwords = stopwords.words('spanish')

import pyLDAvis.gensim_models
from gensim.models import LdaModel
from gensim.corpora import Dictionary

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

D:\Python\anaconda3\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [10]:
# Cargar datos
path = 'C:/Users/XboxL/Documents/01 - Maestria En Analitica de Datos/02 - Procesamiento de Lenguaje Natural (NLP) - Clase/Recursos/reviews_vidjew_es.csv'
data = pd.read_csv(path)
data.head()

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
0,es_0825565,product_es_0370490,reviewer_es_0174781,3,"Buen. Buena calidad, y buena presentación.",Contenta,es,jewelry
1,es_0227934,product_es_0354224,reviewer_es_0411613,3,"Un producto a perfecto, para salir de casa con...",Versatilidad,es,video_games
2,es_0468601,product_es_0665460,reviewer_es_0348315,1,No funciona con Nintendo Switch. No hay forma ...,Decepción absoluta,es,video_games
3,es_0814494,product_es_0692692,reviewer_es_0951508,5,"Recomendado, los utilizo para pc y no me dan n...",Auriculares Pecham ps4,es,video_games
4,es_0206329,product_es_0728826,reviewer_es_0493255,4,El cable funciona bien podria ser un poco mas ...,Perfecto,es,video_games


###  `[12 pts]` Punto 1: Hacer pre-procesamiento del texto

In [11]:
#Eliminacion de columnas
columnasEliminar = ["review_id","product_id","reviewer_id","stars","review_title","language","product_category"]
data = data.drop(columnasEliminar,axis=1)
data.head()

,review_body
0,"Buen. Buena calidad, y buena presentación."
1,"Un producto a perfecto, para salir de casa con..."
2,No funciona con Nintendo Switch. No hay forma ...
3,"Recomendado, los utilizo para pc y no me dan n..."
4,El cable funciona bien podria ser un poco mas ...


In [12]:
#Adicion de columna para el pre-procesamiento
def pre_procesado(texto):
    texto = texto.lower()
    texto = re.sub(r"[\W\d]+", " ", texto)
    texto = [palabra for palabra in texto.split() if palabra not in stopwords]
    return texto 

data['Pre-Processed'] = data['review_body'].apply(lambda texto: pre_procesado(texto))
data.head()


,review_body,Pre-Processed
0,"Buen. Buena calidad, y buena presentación.","[buen, buena, calidad, buena, presentación]"
1,"Un producto a perfecto, para salir de casa con...","[producto, perfecto, salir, casa, nintendo, sw..."
2,No funciona con Nintendo Switch. No hay forma ...,"[funciona, nintendo, switch, forma, emparejarl..."
3,"Recomendado, los utilizo para pc y no me dan n...","[recomendado, utilizo, pc, dan, ningún, proble..."
4,El cable funciona bien podria ser un poco mas ...,"[cable, funciona, bien, podria, ser, mas, larg..."


In [19]:
# Crear una represantación de los documentos en forma de diccionario
dictionary  = Dictionary(data['Pre-Processed'].values)

# Filtrar palabras muy frecuentes o infrecuentes
dictionary .filter_extremes(no_below=5, no_above=0.5) 

# Corpus
corpus = [dictionary.doc2bow(text) for text in data['Pre-Processed'].values]

# Entrenar el modelo
model = LdaModel(corpus = corpus, id2word = dictionary, num_topics = 7, passes=10)

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category,pp
0,es_0825565,product_es_0370490,reviewer_es_0174781,3,"Buen. Buena calidad, y buena presentación.",Contenta,es,jewelry,"[buen, buena, calidad, buena, presentación]"
1,es_0227934,product_es_0354224,reviewer_es_0411613,3,"Un producto a perfecto, para salir de casa con...",Versatilidad,es,video_games,"[producto, perfecto, salir, casa, nintendo, sw..."
2,es_0468601,product_es_0665460,reviewer_es_0348315,1,No funciona con Nintendo Switch. No hay forma ...,Decepción absoluta,es,video_games,"[funciona, nintendo, switch, forma, emparejarl..."
3,es_0814494,product_es_0692692,reviewer_es_0951508,5,"Recomendado, los utilizo para pc y no me dan n...",Auriculares Pecham ps4,es,video_games,"[recomendado, utilizo, pc, dan, ningún, proble..."
4,es_0206329,product_es_0728826,reviewer_es_0493255,4,El cable funciona bien podria ser un poco mas ...,Perfecto,es,video_games,"[cable, funciona, bien, podria, ser, mas, larg..."


###  `[10 pts]` Punto 2: Modelo de LDA

In [27]:
lda_display = pyLDAvis.gensim_models.prepare(model, corpus, dictionary, sort_topics=True)
pyLDAvis.display(lda_display)

###  `[25 pts]` Punto 3: Visualización de LDA

In [24]:
def get_doc_top_n(text_processed, n):
    d = dictionary.doc2bow(text_processed)
    topics = dict(model.get_document_topics(d))
    try:
        return topics[n]
    except:
        return None

for t in range(0,7):
    top_name = f"topic_{t}"
    data[top_name] = data['Pre-Processed'].apply(lambda doc: get_doc_top_n(doc, t))

In [26]:
for t in range(0,7):
    print(f"*********************************** TOPIC {t} ***********************************")
    topic = f"topic_{t}"
    for i,articulo in enumerate(data.sort_values('topic_0', ascending=False)['review_body'].values[:5]):
        print(f"review #{i}")
        print(articulo[:500])
        print()
    print()

*********************************** TOPIC 0 ***********************************
review #0
Es una pena, tenia muchas esperanzas en este juego y se ha convertido en una gran decepción. Los puntos fuertes: La jugabilidad es fantástica, tal como se vió en Red Dead Redemption o Max Payne 3, así como su sistema de coberturas. El sistema de 3 personajes, con opción de cambio rápido, muy logrado. Los gráficos y su rendimiento, sorprende gratamente. Alta calidad y muy pocas bajadas de frames. Ahora viene lo malo, lo decepcionante: Historia pésima, absurda y a un nivel narrativo muy por debajo

review #1
Este producto no esta mal. Si se te ha roto el cargador original, puedes sustituirlo por este. Funciona tanto directamente al puerto tipo C de la switch como a traves del dock original pudiendo en este caso ver la switch en la tv y cargarlo a la vez. "Pero" le he visto un par de inconvenientes; por un lado la clavija esta muy dura al pincharla y da un poco de miedo por si rompes la hembra de ent

###  `[3 pts]` Preguntas

- ¿Qué conclusiones puede sacar de los resultados de los últimos tres talleres, usando diferentes modelos pero los mismos datos?